In [ ]:
!pip install open_clip_torch

In [ ]:
import torch 
import os
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"
# open_clip.list_pretrained()
name_model = 'ViT-SO400M-14-SigLIP-384'
name_dataset = 'webli'

In [ ]:
model,_, preprocess = open_clip.create_model_and_transforms(name_model, device=device, pretrained=name_dataset)
tokenizer = open_clip.get_tokenizer(name_model)

In [ ]:
from PIL import Image
import numpy as np

def get_image(image_path):

    image = Image.open(image_path).convert("RGB")

    return image

get_image(r'/kaggle/input/keyframes-v1-aic2024/Keyframes_L03/keyframes/L03_V002/015.jpg')

In [ ]:
def extract_image_features(image_path):
    image = Image.open(image_path).convert("RGB")
    
    image_input = preprocess(image).to(device).unsqueeze(0)
    
    with torch.no_grad():
        image_features = model.encode_image(image_input)
    
    image_features /= image_features.norm(dim=-1, keepdim=True)
    
    return image_features.cpu().detach().numpy().squeeze().astype(np.float16)

def save_video_features_to_npy(video_features, output_path):
    np.save(output_path, np.array(video_features))

In [ ]:
def process_videos(root_dir, output_dir):
    for keyframes_subdir in sorted(os.listdir(root_dir)):
        keyframes_dir = os.path.join(root_dir, keyframes_subdir, "keyframes")
        
        if os.path.isdir(keyframes_dir): 
            for subdir in sorted(os.listdir(keyframes_dir)):
                video_path = os.path.join(keyframes_dir, subdir)
                
                if os.path.isdir(video_path): 
                    video_features = []
                    
                    for keyframe in sorted(os.listdir(video_path)):
                        keyframe_path = os.path.join(video_path, keyframe)

                        if keyframe.endswith(".jpg"):
                            features = extract_image_features(keyframe_path)
                            video_features.append(features)
                    
                    if video_features:
                        output_npy_path = os.path.join(output_dir, f"{subdir}.npy")
                        os.makedirs(output_dir, exist_ok=True)
                        
                        save_video_features_to_npy(video_features, output_npy_path)
                        print(f"Đã lưu đặc trưng cho video {subdir} trong {keyframes_subdir} vào {output_npy_path}")

In [ ]:
name_folder = 'SigLIP384'

In [2]:
root_dir_b1 = '/kaggle/input/keyframes-v1-aic2024'
output_dir_b1  = f'/kaggle/working/{name_folder}-features-32-b1/{name_folder}-features-32'

process_videos(root_dir_b1, output_dir_b1)